
#**Highway to Hell (Or Heaven)**

                

#1. Introduction(20%)

In a fiercely competitive marketplace, two manufacturing plants, **ShoeCo** and **BootCo**, race to deliver the same product to customers through a shared logistics ecosystem of ports, carriers, warehouse, and retail shops. ShoeCo relies on an efficient, low-cost production process that increases output margins but comes at a heightened risk of defects and BootCo maintains a more expensive yet reliable operation, ensuring higher product quality at greater cost.


As products move through the logistics chain, safety and reliability introduce strict constraints. Warehouses refuse to store any defective goods, enforcing a quality gate before products can proceed downstream. Furthermore, each warehouse supplying a shop must maintain inventory levels exceeding **shop demand** by at least 140%—a buffer designed to guarantee availability even under unpredictable fluctuations in supply or quality losses.


The faster a product reaches the market, the greater its exposure to customers—and the higher its likelihood of being purchased. Plants can choose faster carriers to gain early market access, but rapid transport comes at a premium.

The central challenge of this project is to determine the **optimal logistics path** for each plant through this shared network—selecting carriers, warehouses, and delivery schedules that satisfy quality and inventory constraints while ultimately maximizing total profit. It is a problem not just of routing goods, but of orchestrating risk, time, and cost into a single, winning supply-chain strategy.


- Talk about the use case of these kind of optimisations






<img height=500 align="center" src="https://github.com/ashvin-a/Final-Project-524/blob/main/docs/Logistic-graph.svg?raw=1"/>

This is a depiced graph of the scenario. Here, we are showing the players that are taken into consideration for this optimisation problem. The numbers might not match, for example, we are considering all the ports, i.e,  11 ports, which is considered in the dataset.

# Approach (15%)
We will first solve the optimal path for each plant individually. Then, we will try to make a model which will solve for the optimal path for both the plants. Let's start by analysing the data that is given.

- How the heck are we gonna solve this.
- Be intuitive and cool


In [27]:
# Data Analysis
sheet_names = ["OrderList",
               "FreightRates",
               "WhCosts",
               "WhCapacities",
               "ProductsPerPlant",
               "PlantPorts",
               "VmiCustomers",]

import pandas as pd
df = pd.read_excel("data.xlsx", sheet_name=sheet_names)
df

{'OrderList':           Order ID Order Date Origin Port Carrier  TPT Service Level  \
 0     1.447296e+09 2013-05-26      PORT09   V44_3    1           CRF   
 1     1.447158e+09 2013-05-26      PORT09   V44_3    1           CRF   
 2     1.447139e+09 2013-05-26      PORT09   V44_3    1           CRF   
 3     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 4     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 ...            ...        ...         ...     ...  ...           ...   
 9210  1.447305e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9211  1.447319e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9212  1.447322e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9213  1.447145e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9214  1.447328e+09 2013-05-26      PORT04  V444_1    1           DTD   
 
       Ship ahead day count  Ship Late Day count           Customer  \
 0                        3           

As we can observe, the excel sheet consists of 7 sheets. Here is my description on what each sheets tells us about:

- OrderList: Contains unique identifier as order id.
- FreightRates: Describes all available couriers, the weight gaps for each individual lane and rates associated.
- WhCapacities: Lists warehouse capacities measured in number of orders per day and the WhCosts specifies the cost associated in storing the products in given warehouse measured in dollars per unit.
- ProductsPerPlant: Lists all supported warehouse-product combinations.
- PlantPorts:  Describes the allowed links between the warehouses and shipping ports in real world.
- VmiCustomers: Lists all special cases, where warehouse is only allowed to support specific customer, while any other non-listed warehouse can supply any customer.

In [24]:
for index, dataframe in df.items():
  print(f"\n\n\nTable name: {index}\n Dataframe Summary : {dataframe.columns}\n")
  for column in dataframe.columns:
    if column not in ["Order ID", "Product ID", "minimum cost", "max_wgh_qty", "minm_wgh_qty"]:
      # print(f"Column name : {column}\n ")
      print(f"Column name : {column}\n Unique Entries :{dataframe[column].unique()}")




Table name: OrderList
 Dataframe Summary : Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')

Column name : Order Date
 Unique Entries :<DatetimeArray>
['2013-05-26 00:00:00']
Length: 1, dtype: datetime64[ns]
Column name : Origin Port
 Unique Entries :['PORT09' 'PORT04' 'PORT05']
Column name : Carrier
 Unique Entries :['V44_3' 'V444_0' 'V444_1']
Column name : TPT
 Unique Entries :[1 4 0 2 3]
Column name : Service Level
 Unique Entries :['CRF' 'DTP' 'DTD']
Column name : Ship ahead day count
 Unique Entries :[3 0 5 6 1]
Column name : Ship Late Day count
 Unique Entries :[0 6 1 3]
Column name : Customer
 Unique Entries :['V55555_53' 'V555555555555555_29' 'V5555555_30' 'V55555_2'
 'V555555555555_31' 'V55555555_32' 'V5555555_12' 'V55555_4' 'V555555_6'
 'V5555555555_1' 'V5555_33' '

In [ ]:
for index, dataframe in df.items():
  if index == "OrderList":
    for column in dataframe.columns:
      if column not in ["Order ID", "Product ID", "minimum cost", "max_wgh_qty", "minm_wgh_qty"]:
        print(f"Column name : {column}")

# The Freaking Solution (30%)


In [ ]:
# Data Preprocessing (10%)

In [28]:
!pip install gamspy
!gamspy install license 2a571695-fbd3-4aa6-9587-b3d6083d1be8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 31.9 MB/s eta 0:00:00
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /usr/local/lib/python3.12/dist-packages/gamspy/_cli/install.py:71 in license │
│                                                                              │
│    68 │   is_alp = not os.path.isfile(license)                               │
│    69 │                                                                      │
│    70 │   if is_alp and len(license) != 36:                                  │
│ ❱  71 │   │   raise ValidationError(                                         │
│    72 │   │   │   f"Access code is a 36 character string or an absolute path │
│    73 │   │   │   f"license file but {len(license)} character string ({licen │
│    74 │   │   )                  

In [ ]:
# Do our implementations here

In [ ]:
# Sensitivity Analysis, Visualization (10%)

# Conclusion of the Project

# Extensions of the project
